In [43]:
import collections
import copy
import datetime
import os
from itertools import zip_longest
import pymysql
import dateutil
import numpy as np
import pandas as pd
import sentry_sdk
from scipy import stats

WTS = [1, 1]


In [28]:
def sql_result(query):
    database = pymysql.connect(host="db-read.truebil.com", user="truebil-read",
                               passwd="newreadpassword", db="truebil")
    cursor = database.cursor()
    cursor.execute(query)
    database.commit()
    query_result_list = [row for row in cursor.fetchall()]
    try:
        field_names = [i[0] for i in cursor.description]
        query_result_dataframe = pd.DataFrame(data=query_result_list, columns=field_names)
    except Exception:
        query_result_dataframe = None
    return query_result_dataframe

In [50]:
def get_sold_cars_active_dates(sold_cars_df, price_changes_done):                        
    [sold_data_aDu, sold_data_aDa] = get_active_days_demand(sold_cars_df['id'].tolist()) 
                                                                                         
    sold_data_aDa['latest_dates'] = "{}"                                                 
                                                                                         
    # get the active days of the last active price                                       
    for row in sold_data_aDa.itertuples():                                               
        count = 0                                                                        
        sold_data_aDa.loc[row.Index]['latest_dates'] = dict()                            
        for date in row.dates.keys():                                                
            if date > max(sold_data_aDa.loc[row.Index]['start_date'],                
                          price_changes_done.loc[row.Index]['start_date']):          
                count += 1                                                           
                sold_data_aDa.loc[row.Index]['latest_dates'][date] = row.dates[date] 
                            
    return sold_data_aDa   

def get_price_changes(price_change_query):                                                      
    # get get price changes of all listings                                                     
    price_changes = sql_result(price_change_query)                                              
                                                                                                
    # Last price change done                                                                    
    price_changes_done = price_changes[(price_changes['end_date'] == datetime.date.today()) & ( 
            price_changes['start_date'] != datetime.date.today())].set_index(['listing_id'])    
    return price_changes_done

def instantiate_multi_index_dataframe():
    price_slabs = collections.OrderedDict(
        [('(0, 1)', (0, 1)), ('(1, 3)', (1, 3)), ('(3, 7)', (3, 7)), ('(7, 50)', (7, 50))])

    demand = ['High_Demand', 'Mid_Demand', 'Low_Demand']

    price_slabs_keys = list(price_slabs.keys())[1:]

    stats_metric = ['count', 'mean', 'std']
    cols = pd.MultiIndex.from_product([demand, price_slabs_keys, stats_metric],
                                      names=['model', 'price_slabs_keys', 'metric'])
    return cols, price_slabs

def activeDuration(idList):
    listStr = listToStr(idList)
    result = sql_result(ACTIVE_DURATION_QUERY.format(id_list=listStr))
    return result

def activeDates(actDur, col_start='start_date', col_end='end_date', status='status_to', dates='dates',
                grp=['listing_id'], drp=['active']):
    actDat = actDur.drop(actDur[~actDur[status].isin(drp)].index)
    latest_active_dates = actDat.groupby(grp)[col_start].max()
    actDat.drop([status], axis=1, inplace=True)
    actDat[dates] = actDat.astype('object').apply(lambda x: dateRangeToDateList(x[col_start], x[col_end]),
                                                  axis=1).astype('object')
    actDat.drop([col_start, col_end], axis=1, inplace=True)
    actDat = actDat.groupby(grp).agg(agg_dates_func)
    actDat[dates] = actDat.apply(lambda x: dict.fromkeys(x.dates, 0), axis=1)
    actDat['latest_dates'] = actDat.apply(lambda x: dict(), axis=1)  # new line
    actDat = actDat.merge(latest_active_dates.to_frame(), left_index=True, right_index=True, how="left")
    return actDat

def countLeadsPerDate(inputDf, outputDf, wt):
    for row in inputDf.itertuples():
        outputDf.loc[int(row.listing_id)]['dates'][row.created_at] += wt
        
def lead_genarated(idList, leadList):
    List = []
    if leadList[0]:
        listingLead = listingLeadGenerated(idList)
        List.append(listingLead)
    if leadList[1]:
        offerLead = offerLeadGenerated(idList)
        List.append(offerLead)
    return List

def get_active_days_demand(ids):
    list_all_aDu = activeDuration(ids)
    list_all_aDa = activeDates(list_all_aDu)

    List_Leads = lead_genarated(ids, WTS)

    cou = 0
    for i in range(len(WTS)):
        if WTS[i] != 0:
            countLeadsPerDate(List_Leads[cou], list_all_aDa, wt=WTS[i])
            cou += 1

    return ([list_all_aDu, list_all_aDa])

def get_price_changes(price_change_query):
    # get get price changes of all listings
    price_changes = sql_result(price_change_query)

    # Last price change done
    price_changes_done = price_changes[(price_changes['end_date'] == datetime.date.today()) & (
            price_changes['start_date'] != datetime.date.today())].set_index(['listing_id'])
    return price_changes_done

def listToStr(List, sep=',', pre='(', post=')'):
    return pre + sep.join(str(x) for x in List) + post

def dateRangeToDateList(start, end):
    dates = pd.date_range(start, end)
    return set(x.date() for x in dates)

def agg_dates_func(series):
    return set().union(*series.tolist())

def listingLeadGenerated(idList):
    listStr = listToStr(idList)
    result = sql_result(LISTING_LEAD_GENERATED_QUERY.format(id_list=listStr))
    return result

def offerLeadGenerated(idList):
    listStr = listToStr(idList)
    result = sql_result(OFFER_LEAD_GENERATED_QUERY.format(id_list=listStr))
    return result

In [51]:
SOLD_DATA_QUERY = """SELECT l.id, 
                                  l.price, 
                                  lo.city_id, 
                                  v.model_id 
                            FROM   listings l 
                                  JOIN variants v 
                                        ON l.variant_id = v.id 
                                  JOIN localities lo 
                                        ON l.locality_id = lo.id 
                            WHERE  l.is_inventory = 1 
                                  AND l.created_at >= '{start}' 
                                  AND l.status IN ( 'truebiled' ) 
                                  AND l.price > 10000"""

PRICE_CHANGES_QUERY = """SELECT 
    l.listing_id,
    l.updated_price AS `updated_price`,
    CAST(l.created_at AS DATE) AS start_date,
    CAST(CASE
            WHEN r.created_at IS NULL THEN NOW()
            ELSE r.created_at
        END
        AS DATE) AS end_date
    FROM
    (SELECT 
        @row_no1:=@row_no1 + 1 AS row_no,
            listing_id,
            updated_price,
            created_at, source_update
    FROM
        listing_price_changes, (SELECT @row_no1:=0) t
    ORDER BY listing_id , created_at) l
        LEFT JOIN
    (SELECT 
        @row_no2:=@row_no2 + 1 AS row_no,
            listing_id,
            updated_price,
            created_at, source_update
    FROM
        listing_price_changes, (SELECT @row_no2:=0) p
    ORDER BY listing_id , created_at) r ON l.row_no + 1 = r.row_no
        AND l.listing_id = r.listing_id WHERE {condition} """

ACTIVE_DURATION_QUERY = """SELECT
                   l.listing_id,
                   CAST(l.changed_at AS DATE) start_date,
                   l.status_to,
                   CAST(CASE
                       WHEN r.changed_at IS NULL THEN NOW()
                       ELSE r.changed_at
                   END AS DATE) AS end_date
               FROM
                   (SELECT
                   @row_no1:=@row_no1 + 1 AS row_no,
                       listing_id,
                       status_to,
                       changed_at
               FROM
                   listing_status_track, (SELECT @row_no1:=0) t
               WHERE
                   listing_id IN {id_list}
               ORDER BY listing_id , changed_at) l
               LEFT JOIN (SELECT
                   @row_no2:=@row_no2 + 1 AS row_no,
                       listing_id,
                       status_to,
                       changed_at
               FROM
                   listing_status_track, (SELECT @row_no2:=0) p
               WHERE
                   listing_id IN {id_list} 
               ORDER BY listing_id , changed_at) r ON l.row_no + 1 = r.row_no
                   AND l.listing_id = r.listing_id"""

LISTING_LEAD_GENERATED_QUERY = """SELECT 
                    DISTINCT
                    TB1.id AS buyer_id,
                    listings.id AS listing_id,
                    CAST(buyer_listings.created_at AS DATE) AS created_at
                FROM
                    (SELECT 
                        id, CAST(created_at AS DATE) AS created_at, mobile
                    FROM
                        buyers
                    WHERE
                        mobile NOT IN (SELECT 
                                mobile
                            FROM
                                dnd_numbers)
                            AND mobile NOT IN (SELECT 
                                mobile
                            FROM
                                invalid_numbers)
                            AND mobile NOT IN (SELECT
                                mobile
                            FROM
                                sellers
                            WHERE
                                type_id NOT IN ('Dealer','Subscriber_Dealer'))) AS TB1,
                    buyer_listings,
                    listings
                WHERE
                    buyer_listings.buyer_id = TB1.id
                        AND buyer_listings.listing_id = listings.id
                        AND listings.id IN {id_list} """

OFFER_LEAD_GENERATED_QUERY = """SELECT 
                    DISTINCT
                    TB1.id AS buyer_id,
                    listings.id AS listing_id,
                    CAST(buyer_listing_offers.created_at AS DATE) AS created_at
                FROM
                    (SELECT 
                        id, CAST(created_at AS DATE) AS created_at, mobile
                    FROM
                        buyers
                    WHERE
                        mobile NOT IN (SELECT 
                                mobile
                            FROM
                                dnd_numbers)
                            AND mobile NOT IN (SELECT 
                                mobile
                            FROM
                                invalid_numbers)
                            AND mobile NOT IN (SELECT
                                mobile
                            FROM
                                sellers
                            WHERE
                                type_id NOT IN ('Dealer','Subscriber_Dealer'))) AS TB1,
                    buyer_listing_offers,
                    listings
                WHERE
                    buyer_listing_offers.buyer_id = TB1.id
                        AND buyer_listing_offers.listing_id = listings.id
                        AND buyer_listing_offers.is_verified = 1
                        AND listings.id IN {id_list}  """

In [52]:
price_changes_done = get_price_changes(PRICE_CHANGES_QUERY.format(
            condition=" DATE(l.created_at) NOT IN ('2019-11-18', '2019-10-11', '2019-06-19') "))
sold_data = sql_result(SOLD_DATA_QUERY.format(                                   
    start=datetime.date.today() - dateutil.relativedelta.relativedelta(years=1)))

sold_data_aDa = get_sold_cars_active_dates(sold_cars_df=sold_data, price_changes_done=price_changes_done)

cols, price_slabs = instantiate_multi_index_dataframe()

KeyError: datetime.date(2019, 3, 26)

In [ ]:
def calculate_benchmark_for_z_scores(city_id, sold_data_aDa, cols, price_slabs, sold_data):                                        
    low_demand_price_slab = []                                                                                                     
    cluster_data = pd.DataFrame(columns=cols)                                                                                      
    for price_slab_key in list(price_slabs.keys()):                                                                                
        price_slab = price_slabs[price_slab_key]                                                                                   
        price_wise_listings = sold_data.loc[                                                                                       
            (sold_data.price / 10 ** 5 >= price_slab[0]) & (sold_data.price / 10 ** 5 < price_slab[1]) & (                         
                    sold_data.city_id == city_id)]                                                                                 
                                                                                                                                   
        models = get_demand_model(price_slab)                                                                                      
                                                                                                                                   
        [slab_check, price_wise_listings] = get_slab_check(price_slab, low_demand_price_slab,                                      
                                                           price_wise_listings)                                                    
                                                                                                                                   
        if slab_check == 1 or slab_check == 2:                                                                                     
                                                                                                                                   
            for model_key in list(models.keys()):                                                                                  
                model = models[model_key]                                                                                          
                if price_slab_key == '(7, 50)':                                                                                    
                    model_and_price_wise_listings = price_wise_listings['id']                                                      
                else:                                                                                                              
                    model_and_price_wise_listings = price_wise_listings[price_wise_listings['model_id'].isin(model)]['id']                                     
                                                                                                                                   
                cluster_demand = sold_data_aDa.loc[                                                                                
                    sold_data_aDa.index.intersection(model_and_price_wise_listings)]                                               
                                                                                                                                   
                if cluster_demand.empty:                                                                                           
                    continue                                                                                                       
                                                                                                                                   
                i = 21                                                                                                             
                for slab in DAYS_SLAB.keys():                                                                                      
                                                                                                                                   
                    while True:                                                                                                    
                                                                                                                                   
                        demand_after_last_price_change = cluster_demand.apply(                                                     
                            lambda x: get_seven_days_leads(x.latest_dates, True) if len(x.latest_dates) <= i and len(              
                                x.latest_dates) >= slab[0] else False, axis=1)                                                     
                                                                                                                                   
                        demand_after_last_price_change = demand_after_last_price_change[                                           
                            demand_after_last_price_change != False]                                                               
                        demand_after_last_price_change = demand_after_last_price_change.to_frame().rename(                         
                            columns={0: 'dates'})                                                                                  
                                                                                                                                   
                        demand_dict = demand_after_last_price_change.to_dict()['dates']                                            
                        demand_values_list = []                                                                                    
                                                                                                                                   
                        for listing_row in demand_dict:                                                                            
                            demand_values_list.append(list(demand_dict[listing_row].values()))                                     
                                                                                                                                   
                        if len(demand_values_list) <= 1:                                                                           
                            continue                                                                                               
                                                                                                                                   
                        day_wise_listing_count = []                                                                                
                        for temp_list in demand_values_list:                                                                       
                            day_wise_demand_list = remove_outliers(temp_list)                                                      
                            day_wise_listing_count.append(len(day_wise_demand_list))                                               
                                                                                                                                   
                        i += 1                                                                                                     
                        if day_wise_listing_count[6] > 30 or i > 45:                                                               
                            break                                                                                                  
                                                                                                                                   
                    averages = [np.mean(remove_outliers(day_wise_demand_data)) for                                                 
                                day_wise_demand_data in demand_values_list]                                                        
                    dev = [np.std(remove_outliers(day_wise_demand_data)) for                                                       
                           day_wise_demand_data in demand_values_list]                                                             
                                                                                                                                   
                    if len(day_wise_listing_count) < 7:                                                                            
                        day_wise_listing_count.extend([None] * (7 - len(day_wise_listing_count)))                                  
                    if len(averages) < 7:                                                                                          
                        averages.extend([None] * (7 - len(averages)))                                                              
                    if len(dev) < 7:                                                                                               
                        dev.extend([None] * (7 - len(dev)))                                                                        
                                                                                                                                   
                    for i in range(1, len(dev)):                                                                                   
                        if dev[i] == 0:                                                                                            
                            dev[i] = dev[i - 1]                                                                                    
                                                                                                                                   
                    cluster_data[model_key, price_slab_key, 'count'] = day_wise_listing_count[:7]                                  
                    cluster_data[model_key, price_slab_key, 'mean'] = averages[:7]                                                 
                    cluster_data[model_key, price_slab_key, 'std'] = dev[:7]                                                       
                                                                                                                                   
                    cluster_data = dataframe_type_conversion(cluster_data)                                                         
    cluster_data.to_csv('rolling.csv')                                                                                             
    print("stopppppppppp")                                                                                                         
    return cluster_data                                                                                                            

In [ ]:
calculate_benchmark_for_z_scores(1, sold_data_aDa, cols, price_slabs, sold_data)